In [1]:
%run 10_ngram_common.ipynb

/work/van-speech-nlp/jindaznb/asrenv/lib/python3.10/site-packages/huggingface_hub/utils/_runtime.py:184: UserWarning: Pydantic is installed but cannot be imported. Please check your installation. `huggingface_hub` will default to not using Pydantic. Error message: '{e}'
  warnings.warn(


In [2]:
normal_speakers = ['MC01', 'MC02', 'MC03', 'MC04','FC01','FC02','FC03']
atypical_speakers = ['F01', 'F03', 'F04', 'M01', 'M02', 'M03', 'M04', 'M05']

# read libra

In [3]:
libra = load_dataset("librispeech_asr", 'clean', cache_dir="/work/van-speech-nlp/cache")

/work/van-speech-nlp/jindaznb/asrenv/lib/python3.10/site-packages/datasets/load.py:1454: FutureWarning: The repository for librispeech_asr contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/librispeech_asr
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Loading dataset shards:   0%|          | 0/45 [00:00<?, ?it/s]

In [6]:
translations=libra['train.360']['text']

In [7]:
import nltk
total_tokens = 0

# Iterate over each sentence in the translations corpus
for sentence in translations:
    # Tokenize the sentence into individual words
    tokens = nltk.word_tokenize(sentence)
    # Add the number of tokens in the current sentence to the total count
    total_tokens += len(tokens)

# Print the total number of tokens in the translations corpus
print("Total number of tokens in corpus:", total_tokens)

Total number of tokens in corpus: 3628994


In [8]:
cleaned_translations = []

for text in translations:
    cleaned_text = ' '.join(re.sub(r'[^a-zA-Z0-9]', '', word.lower()) for word in text.split())
    cleaned_translations.append(cleaned_text)

In [9]:
len(translations)

104014

In [10]:
with open("DATASET_libri.txt", "w") as file:
    for text in cleaned_translations:
        file.write(text + "\n")

In [11]:
word_set_europarl = set()

for sentence in cleaned_translations:
    words = sentence.split()
    word_set_europarl.update(words)

In [12]:
len(word_set_europarl)

58354

# train LM with LIBRA

In [21]:
# !kenlm/build/bin/lmplz -o 3 < DATASET_libri.txt > libri.apra --discount_fallback

=== 1/5 Counting and sorting n-grams ===
Reading /work/van-speech-nlp/jindaznb/jslpnb/torgo_error_correction/DATASET_libri.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
Unigram tokens 3595494 types 58357
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:700284 2:55979069440 3:104960753664
Statistics:
1 58357 D1=0.578511 D2=0.988481 D3+=1.43628
2 986830 D1=0.758182 D2=1.0831 D3+=1.37103
3 2458575 D1=0.857684 D2=1.17924 D3+=1.35938
Memory estimate for binary LM:
type    MB
probing 66 assuming -p 1.5
probing 72 assuming -r models -p 1.5
trie    26 without quantization
trie    14 assuming -q 8 -b 8 quantization 
trie    25 assuming -a 22 array pointer compression
trie    13 assuming -a 22 -q 8 -b 8 array pointer compression and quantization
=== 3/5 Calculating and sorting initial probabilities ===
Chain sizes: 1:700

# read torgo

In [15]:
# Read the dataset
data_df = pd.read_csv('torgo_new.csv')
dataset_csv = load_dataset('csv', data_files='torgo_new.csv')

speakers = data_df['speaker_id'].unique()

print(f'Speakers: {", ".join(speakers)}')

Speakers: F01, F03, F04, FC01, FC02, FC03, M01, M02, M03, M04, M05, MC01, MC02, MC03, MC04


In [16]:
import re

# Define a function to clean the text
def clean_text(text):
    # Remove non-alphanumeric characters and convert to lowercase
    cleaned_text = ' '.join(re.sub(r'[^a-zA-Z0-9]', '', word.lower()) for word in text.split())
    return cleaned_text

# Apply the clean_text function to the 'text' column in data_df
data_df['text'] = data_df['text'].apply(clean_text)

In [17]:
data_df['text']

0                                                    stick
1        except in the winter when the ooze or snow or ...
2                                                      pat
3                                                       up
4                                                     meat
                               ...                        
16389                                                 wart
16390          the family requests that flowers be omitted
16391                                                 ship
16392                                                  why
16393                                                 tall
Name: text, Length: 16394, dtype: object

In [19]:
from collections import defaultdict

# Initialize a defaultdict to store the texts for each speaker
speaker_texts = defaultdict(list)

# Iterate through each test speaker
for test_speaker in atypical_speakers:
    valid_speaker = 'F03' if test_speaker != 'F03' else 'F04'
    train_speaker = [s for s in speakers if s not in [test_speaker, valid_speaker]]
    torgo_dataset = DatasetDict()
    torgo_dataset['train'] = dataset_csv['train'].filter(lambda x: x in train_speaker, input_columns=['speaker_id'])
    torgo_dataset['validation'] = dataset_csv['train'].filter(lambda x: x == valid_speaker, input_columns=['speaker_id'])
    torgo_dataset['test'] = dataset_csv['train'].filter(lambda x: x == test_speaker, input_columns=['speaker_id'])

    torgo_dataset['train'] = torgo_dataset['train'].filter(lambda x: x['test_data'] == 0)
    torgo_dataset['validation'] = torgo_dataset['validation'].filter(lambda x: x['test_data'] == 0)
    torgo_dataset['test'] = torgo_dataset['test'].filter(lambda x: x['test_data'] == 1)

    # Filter the DataFrame to get the data for the train split for the selected train speakers
    test_data = torgo_dataset['test']
    
    # Append the texts from the filtered DataFrame to the corresponding speaker in the speaker_texts_train dictionary
    for text in test_data['text']:
        # print(text)
        new_text=clean_text(text)
        # print(new_text)
        speaker_texts[test_speaker].append(new_text)

# # Print the number of texts collected for each speaker
for speaker_id, texts in speaker_texts.items():
    print(f"{speaker_id}: {len(texts)}")
    for text in texts[:10]:
        print(text)
    print()

F01: 126
stick
except in the winter when the ooze or snow or ice prevents
up
know
he slowly takes a short walk in the open air each day
double
you wished to know all about my grandfather
knee
she had your dark suit in greasy wash water all year
car

F03: 592
beta
stubble
stubble
trace
goat
she had your dark suit in greasy wash water all year
well he is nearly ninetythree years old
knew
sip
alpha

F04: 367
fee
yet he still thinks as swiftly as ever
knew
trait
stubble
you wished to know all about my grandfather
chair
trace
go
hair

M01: 407
when he speaks his voice is just a bit cracked and quivers a trifle
trait
trouble
fee
raid
except in the winter when the ooze or snow or ice prevents
a long flowing beard clings to his chin
a long flowing beard clings to his chin
fair
bit

M02: 423
alpha
sip
dagger
fair
park
bubble
bit
well he is nearly ninetythree years old
trouble
chair

M03: 442
now
know
gadget
race
well he is nearly ninetythree years old
well he is nearly ninetythree years old
hai

# calculate unique words

In [20]:
unique_words_per_speaker = {}

for speaker_id, texts in speaker_texts.items():
    unique_words = set()
    for text in texts:
        words = text.split()
        for word in words:
            cleaned_word = re.sub(r'[^a-zA-Z0-9]', '', word.lower())
            if cleaned_word:  
                unique_words.add(cleaned_word) 
    unique_words_per_speaker[speaker_id] = unique_words

for speaker_id, unique_words in unique_words_per_speaker.items():
    print(f"{speaker_id}: {len(unique_words)} unique words")

F01: 117 unique words
F03: 447 unique words
F04: 420 unique words
M01: 381 unique words
M02: 392 unique words
M03: 399 unique words
M04: 283 unique words
M05: 436 unique words


In [21]:
unique_words_per_speaker

{'F01': {'a',
  'about',
  'air',
  'all',
  'alpha',
  'and',
  'as',
  'bat',
  'beard',
  'beta',
  'bit',
  'bubble',
  'bug',
  'car',
  'chair',
  'chin',
  'clings',
  'cracked',
  'dagger',
  'dark',
  'day',
  'double',
  'each',
  'ever',
  'except',
  'fair',
  'fee',
  'feeling',
  'floor',
  'flowing',
  'gadget',
  'giving',
  'go',
  'goat',
  'grandfather',
  'greasy',
  'had',
  'hair',
  'he',
  'him',
  'his',
  'ice',
  'in',
  'is',
  'jacket',
  'jagged',
  'just',
  'knee',
  'knew',
  'know',
  'long',
  'my',
  'observe',
  'of',
  'ooze',
  'open',
  'or',
  'organ',
  'our',
  'park',
  'plays',
  'prevents',
  'pronounced',
  'quivers',
  'race',
  'raid',
  'range',
  'rate',
  'rave',
  'respect',
  'share',
  'she',
  'short',
  'sip',
  'skillfully',
  'slip',
  'slowly',
  'small',
  'snow',
  'spark',
  'speaks',
  'stick',
  'still',
  'stubble',
  'suit',
  'swiftly',
  'swore',
  'takes',
  'the',
  'thinks',
  'those',
  'to',
  'trace',
  'trade',

# vocabulary similarity

In [24]:
out_of_domain_words_per_speaker = {}

for speaker_id, unique_words in unique_words_per_speaker.items():
    overlap = len(unique_words & word_set_europarl)
    similarity = overlap / len(unique_words) if len(unique_words) > 0 else 0
    out_of_domain_similarity = (1 - similarity)*100

    out_of_domain_words = {word for word in unique_words if word not in word_set_europarl}
    out_of_domain_words_per_speaker[speaker_id] = out_of_domain_words

    print(f"{speaker_id}: {out_of_domain_similarity:.2f}% dissimilarity with word_set_europarl")
    print(f"{speaker_id}: out_of_domain_words count: {len(out_of_domain_words)}")
    print(f"Out of domain words for {speaker_id}: {out_of_domain_words}")
    print()

F01: 0.85% dissimilarity with word_set_europarl
F01: out_of_domain_words count: 1
Out of domain words for F01: {'gadget'}

F03: 3.13% dissimilarity with word_set_europarl
F03: out_of_domain_words count: 14
Out of domain words for F03: {'sews', 'sweaters', 'mut', 'foxtrot', 'cupping', 'tango', 'snoop', 'validated', 'advisement', 'gadget', 'ninetythree', 'kilo', 'shimmers', 'exam'}

F04: 2.86% dissimilarity with word_set_europarl
F04: out_of_domain_words count: 12
Out of domain words for F04: {'sweaters', 'mut', 'foxtrot', 'snoop', 'validated', 'advisement', 'gadget', 'ninetythree', 'kilo', 'lilyrare', 'shimmers', 'exam'}

M01: 2.89% dissimilarity with word_set_europarl
M01: out_of_domain_words count: 11
Out of domain words for M01: {'sweaters', 'mut', 'foxtrot', 'snoop', 'validated', 'advisement', 'gadget', 'ninetythree', 'kilo', 'shimmers', 'exam'}

M02: 2.81% dissimilarity with word_set_europarl
M02: out_of_domain_words count: 11
Out of domain words for M02: {'sweaters', 'mut', 'foxtr

# Out of domain analysis

In [26]:
import kenlm

model=kenlm.Model("libri_lm_arpa_files/libri.apra") 
per=model.perplexity("your text sentance")
print(per)

per=model.perplexity("resumption of the session")
print(per)

10139.860730126507
482.3399492347551


Loading the LM will be faster if you build a binary file.
Reading /work/van-speech-nlp/jindaznb/jslpnb/torgo_error_correction/libri_lm_arpa_files/libri.apra
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************


# clean the text

In [28]:
import re

cleaned_speaker_texts = {}

for speaker_id, texts in speaker_texts.items():
    cleaned_texts = []
    for text in texts:
        cleaned_text = ' '.join(re.sub(r'[^a-zA-Z0-9]', '', word.lower()) for word in text.split())
        cleaned_texts.append(cleaned_text)
    cleaned_speaker_texts[speaker_id] = cleaned_texts

for speaker_id, texts in cleaned_speaker_texts.items():
    print(f"Speaker {speaker_id}:")
    for text in texts[:2]:  
        print(text)
    print()

Speaker F01:
stick
except in the winter when the ooze or snow or ice prevents

Speaker F03:
beta
stubble

Speaker F04:
fee
yet he still thinks as swiftly as ever

Speaker M01:
when he speaks his voice is just a bit cracked and quivers a trifle
trait

Speaker M02:
alpha
sip

Speaker M03:
now
know

Speaker M04:
trouble
spark

Speaker M05:
swore
up



In [29]:
# Initialize a dictionary to store perplexities per speaker
perplexities_per_speaker = {}

# Calculate perplexity for each speaker
for speaker_id, texts in cleaned_speaker_texts.items():
    perplexities = []
    for text in texts:
        perplexity = model.perplexity(text)
        perplexities.append(perplexity)
    avg_perplexity = np.mean(perplexities)
    perplexities_per_speaker[speaker_id] = avg_perplexity

# Print average perplexity for each speaker
for speaker_id, avg_perplexity in perplexities_per_speaker.items():
    print(f"Speaker {speaker_id}: Average Perplexity: {avg_perplexity}")

Speaker F01: Average Perplexity: 4475.265114956696
Speaker F03: Average Perplexity: 4212.910971903262
Speaker F04: Average Perplexity: 4414.197808779207
Speaker M01: Average Perplexity: 4248.032533960298
Speaker M02: Average Perplexity: 4310.294530269522
Speaker M03: Average Perplexity: 4282.456091586724
Speaker M04: Average Perplexity: 4326.963071866931
Speaker M05: Average Perplexity: 4499.172864557487


# output unigram

In [44]:
arpa_file = f"libri_lm_arpa_files/libri.apra"
output_file = f"libri_lm_arpa_files/unigram.txt"

unigrams = []

with open(arpa_file, "r") as file:
    start_extraction = False
    for line in file:
        line = line.strip()
        if start_extraction:
            if not line:  
                break
            parts = line.split("\t")
            unigram = parts[1]  
            unigrams.append(unigram)
        elif line.startswith("\\1-grams:"):
            start_extraction = True

with open(output_file, "w") as file:
    for unigram in unigrams:
        file.write(unigram + "\n")

libri.apra
